# DATA 304 – Module 6, Session 1

## Relational Databases with SQLite: Import, Query, Join, Aggregate
**Goal:** Load data into SQLite, practice SQL queries, joins, and aggregation from Python.

**What you'll learn:**
- Create and connect to a SQLite database
- Import CSV data into tables
- Run SELECT, WHERE, ORDER BY, GROUP BY, HAVING
- Perform INNER and LEFT JOINs
- Create indexes and use parameterized queries
- Export results back to CSV

**Prereqs:** Python 3, `pandas`

## 1) Setup

In [1]:
import sqlite3
import pandas as pd
import os

## 2) Create SQLite DB and tables

In [2]:
os.makedirs('./data', exist_ok=True)
db_path = './data/module6_session1.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS employees;')
cur.execute('DROP TABLE IF EXISTS departments;')

cur.execute('''CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT NOT NULL
);''')

cur.execute('''CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name   TEXT NOT NULL,
    dept_id INTEGER,
    salary REAL,
    hire_date TEXT,
    FOREIGN KEY(dept_id) REFERENCES departments(dept_id)
);''')
conn.commit()
print('Tables created.')

Tables created.


## 3) Load CSVs into tables

In [3]:
df_emp = pd.read_csv('data/employees.csv')
df_dep = pd.read_csv('data/departments.csv')

df_dep.to_sql('departments', conn, if_exists='append', index=False) #if_exists → 'fail' (default, error if table exists), 'replace' (drop & recreate), 'append' (insert rows).
df_emp.to_sql('employees', conn, if_exists='append', index=False)

pd.read_sql('SELECT * FROM employees LIMIT 5;', conn)

,emp_id,name,dept_id,salary,hire_date
0,1,Alice,10,90000.0,2021-03-15
1,2,Bob,20,75000.0,2020-07-01
2,3,Carla,10,105000.0,2019-11-20
3,4,Dan,30,68000.0,2022-04-10
4,5,Eve,20,82000.0,2021-12-01


## 4) Basic SELECT, WHERE, ORDER BY

In [4]:
pd.read_sql(''' 
                SELECT emp_id, name, salary
                FROM employees
                ORDER BY salary DESC;
            ''', conn)

,emp_id,name,salary
0,3,Carla,105000.0
1,12,Olivia,102000.0
2,11,Mona,99000.0
3,7,Gina,95000.0
4,1,Alice,90000.0
5,15,Liam,84000.0
6,10,Karen,83000.0
7,5,Eve,82000.0
8,9,Jack,79000.0
9,13,Paul,77000.0


In [5]:
pd.read_sql(''' 
                SELECT name, salary 
                FROM employees 
                WHERE salary >= 80000 
                ORDER BY salary DESC;
            ''', conn)

,name,salary
0,Carla,105000.0
1,Olivia,102000.0
2,Mona,99000.0
3,Gina,95000.0
4,Alice,90000.0
5,Liam,84000.0
6,Karen,83000.0
7,Eve,82000.0


## 5) Aggregation with GROUP BY and HAVING

In [9]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                GROUP BY dept_id
                ORDER BY avg_salary DESC;
            ''', conn)

,dept_id,n_emp,avg_salary
0,40,3,91333.333333
1,10,5,88200.000000
2,30,3,79666.666667
3,20,4,73500.000000


In [10]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                GROUP BY dept_id
                HAVING AVG(salary) >= 80000;
            ''', conn)

,dept_id,n_emp,avg_salary
0,10,5,88200.000000
1,40,3,91333.333333


In [11]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                WHERE salary >=80000
                GROUP BY dept_id;
            ''', conn)

,dept_id,n_emp,avg_salary
0,10,4,90500.0
1,20,1,82000.0
2,30,1,99000.0
3,40,2,98500.0


In [12]:
pd.read_sql(''' 
                SELECT dept_id, COUNT(*) AS n_emp, AVG(salary) AS avg_salary
                FROM employees
                WHERE salary >= 80000
                GROUP BY dept_id
                HAVING AVG(salary) >= 80000;
            ''', conn)

,dept_id,n_emp,avg_salary
0,10,4,90500.0
1,20,1,82000.0
2,30,1,99000.0
3,40,2,98500.0


## 6) Joins

In [13]:
pd.read_sql('''
                SELECT e.emp_id, e.name, d.dept_name, e.salary
                FROM employees e
                INNER JOIN departments d ON e.dept_id = d.dept_id
                ORDER BY e.emp_id;
            ''', conn)

,emp_id,name,dept_name,salary
0,1,Alice,Engineering,90000.0
1,2,Bob,Marketing,75000.0
2,3,Carla,Engineering,105000.0
3,4,Dan,Finance,68000.0
4,5,Eve,Marketing,82000.0
5,6,Fred,Finance,72000.0
6,7,Gina,HR,95000.0
7,8,Irene,Marketing,71000.0
8,9,Jack,Engineering,79000.0
9,10,Karen,Engineering,83000.0


In [14]:
pd.read_sql('''
                SELECT d.dept_id, d.dept_name, e.emp_id, e.name
                FROM departments d
                LEFT JOIN employees e ON e.dept_id = d.dept_id
                ORDER BY d.dept_id;
            ''', conn)

,dept_id,dept_name,emp_id,name
0,10,Engineering,1,Alice
1,10,Engineering,3,Carla
2,10,Engineering,9,Jack
3,10,Engineering,10,Karen
4,10,Engineering,15,Liam
5,20,Marketing,2,Bob
6,20,Marketing,5,Eve
7,20,Marketing,8,Irene
8,20,Marketing,14,Nate
9,30,Finance,4,Dan


## 7) Parameterized queries

In [17]:
min_salary = 80000
sql = 'SELECT name, salary FROM employees WHERE salary >= ? ORDER BY salary DESC;'
pd.read_sql(sql, conn, params=(min_salary,))

,name,salary
0,Carla,105000.0
1,Olivia,102000.0
2,Mona,99000.0
3,Gina,95000.0
4,Alice,90000.0
5,Liam,84000.0
6,Karen,83000.0
7,Eve,82000.0


In [18]:
min_salary = 80000
max_salary = 100000
sql = 'SELECT name, salary FROM employees WHERE salary >= ? AND salary < ? ORDER BY salary DESC;'
pd.read_sql(sql, conn, params=(min_salary, max_salary))

,name,salary
0,Mona,99000.0
1,Gina,95000.0
2,Alice,90000.0
3,Liam,84000.0
4,Karen,83000.0
5,Eve,82000.0


## 8) Export results

In [19]:
res = pd.read_sql('SELECT name, salary FROM employees WHERE salary >= 80000;', conn)
res.to_csv('data/high_paid.csv', index=False)
res.head()

,name,salary
0,Alice,90000.0
1,Carla,105000.0
2,Eve,82000.0
3,Gina,95000.0
4,Karen,83000.0


## 9) Close connection

In [20]:
conn.close()